# Logic

## Inputs
* Get positions with cost
* Get openorders
* Get current market price and last-day's low price

## Processing
### Compute price to close
* Get the DTE
* Compute _price-to-close_ as minumum of *quadratic equation* and last-day's low
* Quadratic equation is:
$103.6008 - 3.63457*x + 0.03454677*x*x$

### At the beginning of every trading day
* For every symbol in position, estimate the _price-to-close_
* Cancel all openorders
* Create open orders with _price-to-close_

### Logic to do the market is open
* Save openorder contracts
* Save filled contracts
* Save openpositions

In [1]:
# Harvesting program
# STATUS: Completed
# Runtime: 10 mins

#***          Start ib_insync (run once)       *****
#___________________________________________________

from ib_insync import *
util.startLoop()
ib=IB().connect('127.0.0.1', 7496, clientId=9) # kavi TWS live
# ib=IB().connect('127.0.0.1', 4001, clientId=9) # kavi IBG live

In [2]:
%%time
import pandas as pd
import numpy as np
import datetime

#******         Paths and variables         ****
#_______________________________________________

datapath = r'./zdata/'
market = 'NSE'

#......     Get the positions   .......
p = ib.positions()

df_positions = pd.DataFrame([(i.contract.conId, i.position, i.avgCost) for i in p], 
             columns=['conId', 'position', 'avgCost'])


#.....     Get the contracts   .......
contracts = [Contract(conId=i.contract.conId, exchange='NSE') for i in p]
qual_contracts = ib.qualifyContracts(*contracts)

# make dataframe of qualified contracts
df_c = util.df([(c['Contract']) for c in util.tree(qual_contracts)])

# to put contracts as a new column in the dataframe, capture it
df_c1 = pd.DataFrame([c for c in qual_contracts], columns=['contract'])

df_contracts = pd.concat([df_c, df_c1], axis=1)

#.....   Get yesterday's ohlc   ........
hist_data = [ib.reqHistoricalData(contract=c, endDateTime='', durationStr='1 D', 
                      barSizeSetting='1 day', whatToShow='BID', useRTH=True) 
for i in range(0, len(qual_contracts), 50)
                    for c in qual_contracts[i:i+50]]

# Get details from the Bar and remove empty [] from the list
hist_list = []
for x in hist_data:
    if x != []:
        for h in x:
            hist_list.append((h.date, h.open, h.high, h.low, h.close))
    else:
        hist_list.append(0)

# make yesterday's dataframe
cols=['D', 'O', 'H', 'L', 'C']
df_yesterday = pd.DataFrame([x 
                             if isinstance(x, tuple) 
                             else (x, x) 
                             for x in hist_list], 
                            columns=cols)

#.....    Merge the dataframes and get target price  ....
df1 = pd.concat([df_positions, df_yesterday], axis=1)

df2 = df1.merge(df_contracts,how='left', on='conId')

# get the DTE
df2['DTE'] = (pd.to_datetime(df2.lastTradeDateOrContractMonth) - datetime.datetime.now()).dt.days

# get expected price from DTE
def get_expPct(x):
    '''Gets the expected price percentage from DTE
    Assumes max DTE to be 30 days
    Arg: (x) as float
    Returns: expPrice as float
    Ref: http://interactiveds.com.au/software/Linest-poly.xls
    '''
    if x > 30:
        x = 30  # Forces the max DTE to be 30
    
    expPrice = 103.6008 - 3.63457*x + 0.03454677*x*x
    return expPrice/100

df2['expPrice'] = df2.DTE.apply(get_expPct)*df2.avgCost

# Set target price to be lower than yesterday's low and expPrice
# ...rounded to 0.05
df2['tgtPrice'] = df2[['L', 'expPrice']].min(axis=1).apply(lambda x: round(x*2,1)/2)

df2['order'] = [LimitOrder(action='BUY',totalQuantity=-position, lmtPrice=tgt_price)
            for position, tgt_price in zip(df2.position, df2.tgtPrice)]

Started to throttle requests
Stopped to throttle requests


Wall time: 7min 6s


In [3]:
len(df2)

181

In [ ]:
#------------------------------------------------
# WARNING: THIS SECTION PLACES CLOSING TRADES
#________________________________________________

In [ ]:
# ....    Cancel all openorders   .....
cancels = ib.reqGlobalCancel()

In [4]:
# ....   Place closing trades    .....
closingTrade = [ib.placeOrder(contract, order) for contract, order in zip(df2.contract, df2.order)]

Started to throttle requests
Error 463, reqId 1866: You must enter a valid price.
Canceled order: Trade(contract=Contract(secType='OPT', conId=335383983, symbol='RCOM', lastTradeDateOrContractMonth='20181227', strike=9.0, right='P', multiplier='1', exchange='NSE', currency='INR', localSymbol='RCOM18DEC9PE', tradingClass='RCOM'), order=LimitOrder(orderId=1866, clientId=9, action='BUY', totalQuantity=28000.0, lmtPrice=0.0), orderStatus=OrderStatus(status='Cancelled'), fills=[], log=[TradeLogEntry(time=datetime.datetime(2018, 12, 6, 16, 17, 14, 621237, tzinfo=datetime.timezone.utc), status='PendingSubmit', message=''), TradeLogEntry(time=datetime.datetime(2018, 12, 6, 16, 17, 16, 578443, tzinfo=datetime.timezone.utc), status='Cancelled', message='Error 463, reqId 1866: You must enter a valid price.')])
Stopped to throttle requests


In [5]:
# Write closed orders placed to file
x  =  datapath + datetime.datetime.now().strftime("%Y%m%d_%H%M%S") + "_closing_orders.xlsx"
writer = pd.ExcelWriter(x)
df2.to_excel(writer, 'all-options', index=False, header=1)
writer.save()

In [6]:
ib.disconnect()